In [1]:
import torch

In [2]:
x= torch.empty(5, 4)

In [3]:
y = torch.rand(4,3)

In [4]:
x.size()

torch.Size([5, 4])

In [5]:
y.numpy()

array([[0.38652968, 0.44365197, 0.5621989 ],
       [0.61431617, 0.7587349 , 0.09436256],
       [0.31840605, 0.8379941 , 0.60013336],
       [0.11389869, 0.8971702 , 0.67891383]], dtype=float32)

In [6]:
torch.matmul(x, y)

tensor([[-3.3983e-27, -6.1786e-27, -5.6034e-27],
        [-1.1491e-27, -2.8697e-27, -2.1211e-27],
        [ 0.0000e+00,  0.0000e+00,  1.4013e-45],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00]])

# AutoGrad

In [7]:
a = torch.ones(2, 2, requires_grad=True)

In [8]:
b = a+2

In [9]:
z = 2*b*b+3*b+1

In [10]:
z

tensor([[28., 28.],
        [28., 28.]], grad_fn=<AddBackward0>)

In [11]:
z.grad_fn

In [12]:
out = z.mean()

In [13]:
out.backward()

In [14]:
# gradients d(out)/da
a.grad

tensor([[3.7500, 3.7500],
        [3.7500, 3.7500]])

# CNN

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [17]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [18]:
# net will take in a 4D Tensor of nSamples x nChannels x Height x Width
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.1229,  0.0830,  0.0834, -0.0489,  0.0367, -0.0392,  0.1257,  0.0452,
         -0.1707,  0.0066]], grad_fn=<AddmmBackward>)


In [19]:
# Loss Function
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(1.1019, grad_fn=<MseLossBackward>)


In [20]:
# Backprop
net.zero_grad()     # zeroes the gradient buffers of all parameters
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)
loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0088,  0.0186,  0.0035,  0.0199,  0.0036, -0.0038])


In [21]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update